In [1]:
import os
import requests
import json
import openai

import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer

/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [3]:
# df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [3]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [4]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [26]:
with open('data/ram_tag_list.txt', 'r') as f:
    check=f.readlines()

In [27]:
len(check)

4585

In [29]:

check=[x.split('\n')[0] for x in check]


In [36]:
maf=check

In [37]:
len(maf)

4585

In [38]:
call=iab_labels+maf

In [39]:
len(call)

4745

In [40]:
print(call[0:10])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel']


In [41]:
total_embeddings=[]

In [42]:
def create_embeddings(item,total_embeddings):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

    

In [43]:
for items in tqdm(call):
    create_embeddings(items,total_embeddings)

100%|██████████| 4745/4745 [11:14<00:00,  7.03it/s]


In [44]:
len(total_embeddings)

4745

In [45]:
iab_embeddings=total_embeddings[0:160]

In [46]:
maf_embeddings=total_embeddings[160:]

In [47]:
def calculate_precision_recall(true_labels, predictions):
    true_positives = sum(1 for pred in predictions if pred in true_labels)
    false_positives = len(predictions) - true_positives
    false_negatives = len(true_labels) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

In [48]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [52]:
nodes=maf
#node_id=df_MAF['node_id'].to_list()
#IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
#H_Mappings=df_MAF['H_Scores'].to_list()

In [53]:
cos_sim=cosine_similarity(np.array(iab_embeddings[90]).reshape(1,-1),np.array(maf_embeddings[15]).reshape(1,-1))
print(cos_sim)

[[0.80021651]]


In [54]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_node_ids=[]
top_n_IAB_Candidates=[]
top_n_H_scores=[]
top_n_labels=[]
top_n_cosines=[]

for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+i],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)


    #threshold logic needed
    similarities=[x for x in similarities if x[2]>=0.765]

    
    
    if len(similarities)==0:
        similarities.append(['No_Pred','No_Pred',[[0]]])
    
    temp_iab_labels=[]
    temp_cosines=[]


    for items in similarities:
        #print(items[1])
        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])
        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0])
        top_n_nodes.append(nodes[i])
        #top_n_node_ids.append(node_id[i])
        #top_n_IAB_Candidates.append(IAB_Candidates[i])
        #top_n_H_scores.append(H_Mappings[i])


    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)


100%|██████████| 4585/4585 [05:47<00:00, 13.20it/s]


In [55]:
len(top_n_labels)

4585

In [56]:
len(top_n_labels_new)

307967

In [57]:
len(top_n_cosines)

4585

In [122]:
MAF_predictions=df_MAF['IAB_Candidates'].to_list()

In [123]:
candidate_count=0
hit_count=0
precision=0
recall=0
count=0
f1=0

for predictions, labels in zip(MAF_predictions,top_n_labels):
    count+=1
    proper=ast.literal_eval(predictions)
    prec, reca=calculate_precision_recall(predictions, labels)

    f=calculate_f1_score(prec, reca)
    precision+=prec
    recall+=reca
    f1+=f


    candidate_count+=len(proper)
    for items in labels:
        if items in proper:
            hit_count+=1

In [124]:
hit_count/candidate_count

0.7809958341598889

In [125]:
precision/942

0.0779960795886409

In [126]:
recall/942

0.12264216664289772

In [127]:
f1/942

0.0790254846133288

In [68]:
dic={'item':top_n_nodes, 
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new}

In [69]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [ ]:
df_MAF_new.to_csv('results/RAM_results_0.765_newmapping.csv')

In [61]:
#Average adwords per item
count=0
for items in tqdm(maf):
    count+=df_MAF_new[df_MAF_new['item']==items].shape[0]

print(count/len(maf))

100%|██████████| 4585/4585 [01:02<00:00, 73.08it/s]

67.16837513631407


In [62]:
counts=[]
for items in tqdm(iab_labels):
    lol=df_MAF_new[df_MAF_new['predictions']==items].shape[0]
    counts.append(lol)

dic={'IAB_Label':iab_labels, 'Count':counts}

df_adwords=pd.DataFrame.from_dict(dic)
df_adwords.to_csv('results/Adwords_Statistics.csv',index=False)

  0%|          | 0/160 [00:00<?, ?it/s]

100%|██████████| 160/160 [00:02<00:00, 75.26it/s]


In [65]:
iab_labels[counts.index(max(counts))]

'Beauty'

In [63]:
iab_labels[counts.index(min(counts))]

'Healthcare Industry'

In [64]:
sum(counts)/160

1924.775

In [132]:
print(count/len(maf))

65.47014925373135


In [87]:
df_MAF_new[df_MAF_new['item']=='crane'].shape[0]

157

In [84]:
df_MAF_new.shape

(232289, 4)

In [ ]:
#Top 100 in predictions but not in embedding

In [108]:
df_MAF_new=df_MAF_new.sort_values(by='cosine_predicted', ascending=False)

In [109]:
df_MAF_new.iloc[0]

item                                       minivan
node_id                                       4867
IAB_Candidates      ['Auto Parts', 'Auto Rentals']
predictions                                Minivan
cosine_predicted              [0.9860634986725173]
H_Mapping_scores        [0.724198355, 0.730903592]
Name: 37363, dtype: object

In [110]:
top_100_node=[]
top_100_embedding_pred=[]
top_100_H_pred=[]
top_100_cos_pred=[]
top_100_H_score=[]

In [111]:
items=df_MAF_new['item'].to_list()
IAB_Candidates=df_MAF_new['IAB_Candidates'].to_list()
predictions=df_MAF_new['predictions'].to_list()
cosine_predicted=df_MAF_new['cosine_predicted'].to_list()
H_Mapping_scores=df_MAF_new['H_Mapping_scores'].to_list()

In [112]:
for item,IAB,pred,cos,H_map in tqdm(zip(items,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores)):
    pred_list=ast.literal_eval(IAB)
    if pred not in pred_list:
        top_100_node.append(item)
        top_100_embedding_pred.append(pred)
        top_100_H_pred.append(IAB)
        top_100_cos_pred.append(cos)
        top_100_H_score.append(H_map)
    if len(top_100_node)==100:
        break

253it [00:00, 19364.57it/s]


In [113]:
top_100_dic={'items':top_100_node, 'Embedding Predictions': top_100_embedding_pred,
 "H_Mapping_Pred": top_100_H_pred, 'Predicted Cosine': top_100_cos_pred, 
 'H Mapping Cosine': top_100_H_score}

In [114]:
df_top_100=pd.DataFrame.from_dict(top_100_dic)

In [115]:
df_top_100.to_excel('results/OpenAI_Top_100_Not_In_H.xlsx', index=False)

In [ ]:
#Top 100 in H but not in Embedding

In [116]:
print(top_n_labels[5])
print(top_n_cosines[5])

['Dogs', 'SUV', 'Cats', 'Large Animals', 'Sedan', 'Pets', 'Polar Travel', 'Camping', 'Beauty', 'Desktops', 'Wellness', 'Science', 'Minivan', 'Reptiles', 'Pickup Trucks', 'Laptops', 'Sports', 'Travel', 'Outdoor Activities', 'Hair Care', 'Hobbies & Interests', 'Birds', 'Cruises', 'Parenting', 'Dating', 'Convertible', 'Games and Puzzles', 'Movies', 'Shaving', 'Smartphones', 'Motorcycles', 'Shopping', 'Graduation', 'Cooking', 'Parks & Nature', 'Holiday Shopping', 'Space and Astronomy', 'Luxury Cars', 'Healthy Living', 'Single Life', 'Education', 'Commercial Trucks', 'Rail Travel', 'Houses', 'Station Wagon', 'Wedding', 'Travel Locations', "Women's Handbags and Wallets", 'Coupe', 'Travel Accessories', 'Beach Travel', 'Apartments', 'Grocery Shopping', 'Asia Travel', 'South America Travel', 'Board Games and Puzzles', 'Barbecues and Grilling', 'Bath and Shower', 'Europe Travel', 'Landscaping', 'Fitness and Exercise', 'Home Security', 'Smart Home', 'Air Travel', 'Home & Garden', 'Gardening', 'Zo

In [117]:
nodes=df_MAF['item'].to_list()
# node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
H_Mappings=df_MAF['H_Scores'].to_list()

In [118]:
mid_node=[]
mid_IAB=[]
mid_label=[]
mid_cos=[]
mid_H_map=[]

for node, IAB, label, cos, H_map in tqdm(zip(nodes,IAB_Candidates,top_n_labels,top_n_cosines,H_Mappings)):
    IAB_list=ast.literal_eval(IAB)
    H_Map_list=ast.literal_eval(H_map)
    for IAB_item, H_Map_item in zip(IAB_list,H_Map_list):
        mid_node.append(node)
        mid_IAB.append(IAB_item)
        mid_label.append(label)
        mid_cos.append(cos)
        mid_H_map.append(H_Map_item)

942it [00:00, 34295.68it/s]


In [119]:
dic={'item':mid_node, 'IAB_Candidates': mid_IAB,
'predictions': mid_label, 'cosine_predicted': mid_cos, "H_Mapping_scores": mid_H_map}

In [120]:
df_lol_100=pd.DataFrame.from_dict(dic)

In [121]:
df_lol_100=df_lol_100.sort_values(by='H_Mapping_scores', ascending=False)

In [122]:
df_lol_100.head()

,item,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores
2613,wagon,Travel,"[Station Wagon, Minivan, SUV, Convertible, Tra...","[[0.8676690064930836], [0.8482987892336509], [...",1.0
2614,drop,Real Estate,"[Dating, Travel, Graduation, Movies, Shopping,...","[[0.8176049105530151], [0.8087438873386152], [...",1.0
2615,drop,Travel,"[Dating, Travel, Graduation, Movies, Shopping,...","[[0.8176049105530151], [0.8087438873386152], [...",1.0
2629,mousetrap,Home & Garden,"[Cats, Movies, Coupe, Birds, Games and Puzzles...","[[0.7973278023641381], [0.7922019973205039], [...",1.0
2626,ring_mail,Events and Attractions,"[Rail Travel, Travel, Birds, Nail Care, Hair C...","[[0.7915800550643299], [0.7658795812872278], [...",1.0


In [123]:
top_100_node=[]
top_100_embedding_pred=[]
top_100_H_pred=[]
top_100_cos_pred=[]
top_100_H_score=[]

In [124]:
items=df_lol_100['item'].to_list()
IAB_Candidates=df_lol_100['IAB_Candidates'].to_list()
predictions=df_lol_100['predictions'].to_list()
cosine_predicted=df_lol_100['cosine_predicted'].to_list()
H_Mapping_scores=df_lol_100['H_Mapping_scores'].to_list()

In [125]:
for item,IAB,pred,cos,H_map in tqdm(zip(items,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores)):
    #pred_list=ast.literal_eval(pred)
    if IAB not in pred:
        top_100_node.append(item)
        top_100_embedding_pred.append(pred)
        top_100_H_pred.append(IAB)
        top_100_cos_pred.append(cos)
        top_100_H_score.append(H_map)
    if len(top_100_node)==100:
        break

1311it [00:00, 678855.87it/s]


In [126]:
top_100_dic={'items':top_100_node, 'Embedding Predictions': top_100_embedding_pred,
 "H_Mapping_Pred": top_100_H_pred, 'Predicted Cosine': top_100_cos_pred, 
 'H Mapping Cosine': top_100_H_score}

In [127]:
df_top_100=pd.DataFrame.from_dict(top_100_dic)

In [128]:
df_top_100.to_excel('results/Top_100_Not_In_OpenAI.xlsx', index=False)